In [1]:
import cv2
import torch
import os
import numpy as np

In [6]:
finger_model = torch.hub.load('ultralytics/yolov5', 'custom','./best.pt')
finger_model.conf = 0.5

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\Ahmed Ayman/.cache\torch\hub\master.zip
YOLOv5  2022-1-23 torch 1.10.0 CUDA:0 (NVIDIA GeForce GTX 1050, 4096MiB)

Fusing layers... 
Model Summary: 280 layers, 12308200 parameters, 0 gradients, 16.2 GFLOPs
Adding AutoShape... 


In [7]:
def finger_detector(img):
    h, w, c = img.shape
    results = finger_model(img, size=h)
    detections = results.xyxy[0].cpu().numpy()
    return detections

In [8]:
folderPath = "design/font"
myList = os.listdir(folderPath)
fontList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    image = cv2.resize(image, (60,400), interpolation = cv2.INTER_AREA)
    fontList.append(image)
    
font= fontList[3]

In [9]:
folderPath = "design/left"
myList = os.listdir(folderPath)
overlayList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    image = cv2.resize(image, (75,720), interpolation = cv2.INTER_AREA)
    overlayList.append(image)
    
left = overlayList[0]

In [10]:
folderPath = "design/top"
myList = os.listdir(folderPath)
colorList = []
for imPath in myList:
    image = cv2.imread(f'{folderPath}/{imPath}')
    image = cv2.resize(image, (600,60), interpolation = cv2.INTER_AREA)
    colorList.append(image)
    
top = colorList[0]

In [11]:
drawColor = (255, 0, 255)
cap = cv2.VideoCapture(0)

cap.set(3, 1280)
cap.set(4, 720)

brushThickness = 5
eraserThickness = 100
xp, yp = 0, 0
imgCanvas = np.zeros((720, 1280, 3), np.uint8)

while True:
    success, img = cap.read()
    img=cv2.flip(img, 1)
    
    detections = finger_detector(img)

    
    # If Drawing Mode - Index finger is up
    if len(detections) == 1 :
        x1, y1 = round(detections[0][0] +20), round(detections[0][1] +20)
        
        cv2.circle(img, (x1, y1), 15, drawColor, cv2.FILLED)
        if xp == 0 and yp == 0:
            xp, yp = x1, y1
 
        cv2.line(img, (xp, yp), (x1, y1), drawColor, brushThickness)
        cv2.line(imgCanvas, (xp, yp), (x1, y1), drawColor, brushThickness)
 
        xp, yp = x1, y1
    
    

    if len(detections) == 2 :
        x1, y1 = round(detections[0][0] +20), round(detections[0][1] +20)
        x2, y2 = round(detections[1][0] +20), round(detections[1][1] +20)
        xp, yp = x1, y1
                    #change color 
        
        if y1 < 60:
            if 340 < x1 < 590:
                top = colorList[0]
                drawColor = (255, 0, 255)
            elif 590 < x1 < 740:
                top = colorList[1]
                drawColor = (255, 0, 0)
                
            elif 740 < x1 < 900 :
                top = colorList[2]
                drawColor = (0, 255, 0)
                
                    #change font                
        if  1180 < x1 < 1280 or 1180 < x2 < 1280:
            
            if  160 < y1 < 260 or 160 < y2 < 260:
                font= fontList[3]
                brushThickness = 10

            elif 260 < y1 < 360 or 260 < y2 < 360:
                font= fontList[2]
                brushThickness = 25

            elif 360 < y1 < 460 or 360 < y2 < 460:
                font= fontList[1]
                brushThickness = 50

            elif 460 < y1 < 560 or 460 < y2 < 560:
                font= fontList[0]
                brushThickness = 100
                
                
        
    # If Erasing Mode
    if 5 > len(detections) >= 3 :
        x1, y1 = round(detections[0][0] +20), round(detections[0][1] +20)
        x2, y2 = round(detections[1][0] +20), round(detections[1][1] +20)
        cv2.rectangle(img, (x1, y1 - 25), (x2, y2 + 25), (0, 0, 0), cv2.FILLED)
        cv2.line(img, (xp, yp), (x1, y1), (0, 0, 0), eraserThickness)
        cv2.line(imgCanvas, (xp, yp), (x1, y1), (0, 0, 0), eraserThickness)
        xp, yp = x1, y1
        
    if len(detections) == 5 :
        imgCanvas = np.zeros((720, 1280, 3), np.uint8)

    
    
    imgGray = cv2.cvtColor(imgCanvas, cv2.COLOR_BGR2GRAY)
    _, imgInv = cv2.threshold(imgGray, 50, 255, cv2.THRESH_BINARY_INV)
    imgInv = cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
    img = cv2.bitwise_and(img,imgInv)
    img = cv2.bitwise_or(img,imgCanvas)
    
    
    img[0:720, 0:75] = left
    
    img[160:560 , 1220:1280]=font
    
    img[0:60 , 340:940]=top
    
    cv2.imshow("Image", img)
#     cv2.imshow("Canvas", imgCanvas)
    #     cv2.imshow("Inv", imgInv)
    
    cv2.waitKey(1)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
                break
            
            
cap.release()
cv2.destroyAllWindows()

In [12]:
conda list -e > requirements.txt


Note: you may need to restart the kernel to use updated packages.
